In [ ]:
! yes | . ./setup.sh

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import pickle5 as pickle
import os
import time
import json

In [2]:
start = time.perf_counter()

In [3]:
dirlist = ['models', 'data']

for dir_ in dirlist:
    try:
        os.makedirs(dir_)
    except:
        print(f'Could not make directory {dir_}')

#### Load longitude and latitude reference

In [4]:
# coords_dict = {}

# for pkl in os.listdir('.'):
#     if pkl.endswith('pkl'):
#         with open(pkl, 'rb') as f:
#             pkldata = pickle.load(f)
#             coords_dict.update(pkldata)
#         f.close()

In [5]:
# coords_dict = pickle.load(open('add_to_geo_map.pkl', 'rb'))

In [6]:
coords_dict = json.load(open('add_to_geo_map.json' , 'r'))

#### Connecting to the Database
First we need to connect to the database with the data.

In [7]:
%%time
# Connect to Postgres
import os
import psycopg2
def newCursor():
    try:
        connection = psycopg2.connect(user = os.environ["DB_USER"],
                                      host = os.environ["DB_HOST"],
                                      password = os.environ["DB_PASSWORD"],
                                      port = "5432",
                                      database = os.environ["DB_DATABASE"])

        cursor = connection.cursor()
        return cursor

    except (Exception, psycopg2.Error) as error:
        print("Error while connecting to PostgreSQL", error)
        return error

CPU times: user 6.09 ms, sys: 7.17 ms, total: 13.3 ms
Wall time: 12.3 ms


#### Get the query result as a dataframe

In [8]:
# import pandas as pd

# c = newCursor()
# q = """
# SELECT
# -- regexp_matches(":32A:", '(\A[0-9]+)([A-Z][A-Z][A-Z])([0-9]+,[0-9]+)', 'g')
# -- regexp_matches(":32A:", '(\A[0-9]+).', 'g') as date
# FROM swift."simulated_fin_parsed_hackathon"
# LIMIT 10
# """
# c.execute(q)
# cols = [desc[0] for desc in c.description]
# df = pd.DataFrame(c.fetchall(), columns=cols)
# df.head(20)

In [9]:
%%time
import pandas as pd

c = newCursor()
q = """
SELECT *
FROM swift."simulated_fin_parsed_hackathon"
-- LIMIT 10
"""
c.execute(q)
cols = [desc[0] for desc in c.description]
df = pd.DataFrame(c.fetchall(), columns=cols)
df.head()

CPU times: user 2.45 s, sys: 545 ms, total: 3 s
Wall time: 3.08 s


,UETR:,Sender,Receiver,:20:,:23B:,:32A:,:33B:,:36:,:50A:,:50F:,:50K:,:52A:,:56A:,:57A:,:59F:,:71A:,:71F:,:71G:,:79:,Type
0,bc95b4fb-fcf3-4d9f-a7b8-c641b16913ef,BIC68DF2,BIC6A94A,FIN001-NO-FX-679,CRED,"210813RUB1544103,68","RUB1544103,68",None,/0000000000000000000000000000009621\n BIC68DF2C94,None,None,BIC68DF2C94,None,BIC6A94AE24,/0000000000000000000000000000006099\n 1/ANGEL ...,SHA,None,None,None,103
1,d4b46ac4-e22b-47e5-8385-335699c41166,BIC68DF2,BIC6A94A,FIN001-NO-FX-3374,CRED,"210813RUB912070,95","RUB912070,95",None,/0000000000000000000000000000007346\n BIC68DF2C94,None,None,BIC68DF2C94,None,BIC6A94AE24,/0000000000000000000000000000000874\n 1/JAKUB ...,BEN,"RUB0,00",None,None,103
2,758bd6ce-526d-4a08-a179-10ca089053b9,BIC68DF2,BIC6A94A,FIN001-NO-FX-6478,CRED,"210813RUB21760302,59","RUB21760302,59",None,/0000000000000000000000000000005305\n BIC68DF2C94,None,None,BIC68DF2C94,None,BIC6A94AE24,/0000000000000000000000000000005436\n 1/ZIGA M...,SHA,None,None,None,103
3,f435c830-a673-482a-83f4-74ded26ec666,BIC68DF2,BIC6A94A,FIN001-NO-FX-11185,CRED,"210813RUB878587,53","RUB878587,53",None,/0000000000000000000000000000004127\n BIC68DF2C94,None,None,BIC68DF2C94,None,BIC6A94AE24,/0000000000000000000000000000008707\n 1/YAEL D...,SHA,None,None,None,103
4,22429a61-5fad-4481-b75f-e5fd8a241215,BIC68DF2,BIC6A94A,FIN001-NO-FX-11284,CRED,"210813RUB6696355,97","RUB6696355,97",None,/0000000000000000000000000000006978\n BIC68DF2C94,None,None,BIC68DF2C94,None,BIC6A94AE24,/0000000000000000000000000000004326\n 1/NOEL P...,SHA,None,None,None,103


In [10]:
print(df[df[':50A:'] != ""].shape)
print(df[df[':50F:'] != ""].shape)
print(df[df[':50K:'] != ""].shape)

(379526, 20)
(379526, 20)
(379526, 20)


#### Remove nan values

In [11]:
%%time
import numpy as np
import re

df = df.replace(np.nan, '', regex=True)

CPU times: user 583 ms, sys: 27.2 ms, total: 610 ms
Wall time: 607 ms


#### Create a new dataframe for transformed attributes

In [12]:
dfm = pd.DataFrame()

In [13]:
import concurrent.futures
from functools import partial

In [14]:
# simple wrapper code around serial_calc to parallelize the work
def parallel_calc(df, func, n_core, col):
    futs = []
    df_split = np.array_split(df, n_core)
    # pool = concurrent.futures.ThreadPoolExecutor(max_workers = n_core)
    pool = concurrent.futures.ProcessPoolExecutor(max_workers = n_core)
    apply_partial = partial(func, col=col)
    return pd.concat(pool.map(apply_partial, df_split))

#### Create DATETIME field

In [15]:
%%time
def datefunc(row, col):
    return pd.to_datetime(re.search('([0-9]+)?', row[col]).group(0), 
                             format='%y%m%d', 
                             errors='ignore')

def serial_proc_datefunc(df, col):
    apply_partial = partial(datefunc, col=col)
    df['32a_val_dt'] = df.apply(apply_partial, axis=1)
    return df

dfm['32a_val_dt'] = parallel_calc(df, serial_proc_datefunc, 8, ':32A:')['32a_val_dt']
# dfm['32a_val_dt'] = serial_proc_datefunc(df, ':32A:')['32a_val_dt']

CPU times: user 3.11 s, sys: 1.23 s, total: 4.34 s
Wall time: 24.9 s


In [16]:
dfm

,32a_val_dt
0,2021-08-13
1,2021-08-13
2,2021-08-13
3,2021-08-13
4,2021-08-13
...,...
379521,2021-08-13
379522,2021-08-13
379523,2021-08-13
379524,2021-08-13


In [17]:
# %%time
# dfm['32a_val_dt_yr'] = pd.DatetimeIndex(dfm['32a_val_dt']).year
# dfm['32a_val_dt_mth'] = pd.DatetimeIndex(dfm['32a_val_dt']).month
# dfm['32a_val_dt_day'] = pd.DatetimeIndex(dfm['32a_val_dt']).day

#### Split 33B into Currency and Amount

In [18]:
%%time
def _33bcurAmt(row, col):
    return pd.Series(re.split("(^[A-Z][A-Z][A-Z])([0-9]*,[0-9]*)", row[col])[1:3], 
                                index=[['cur', 'amt']])

def serial_proc_33bcurAmt(df, col):
    apply_partial = partial(_33bcurAmt, col=col)
    df[['33b_cur', '33b_orig_ord_amt']] = df.apply(apply_partial, axis=1)
    return df

dfm[['33b_cur', '33b_orig_ord_amt']] = parallel_calc(df, serial_proc_33bcurAmt, 8, ':33B:')[['33b_cur', '33b_orig_ord_amt']]
# dfm[['33b_cur', '33b_orig_ord_amt']] = serial_proc_33bcurAmt(df, ':33B:')[['33b_cur', '33b_orig_ord_amt']]

CPU times: user 3.84 s, sys: 1.43 s, total: 5.27 s
Wall time: 2min 19s


In [19]:
dfm

,32a_val_dt,33b_cur,33b_orig_ord_amt
0,2021-08-13,RUB,"1544103,68"
1,2021-08-13,RUB,"912070,95"
2,2021-08-13,RUB,"21760302,59"
3,2021-08-13,RUB,"878587,53"
4,2021-08-13,RUB,"6696355,97"
...,...,...,...
379521,2021-08-13,USD,"8339,00"
379522,2021-08-13,USD,"4001,56"
379523,2021-08-13,USD,"4456,00"
379524,2021-08-13,USD,"8836,10"


#### 50A Payor Lattitude and Longitude

In [20]:
%%time
def _50aPayor(row, col):
    return pd.Series(row[col].split("\n") if len(row[col]) > 0 else [''] * 2,
                                  index=['lat', 'lon'])

def serial_proc_50aPayor(df, col):
    apply_partial = partial(_50aPayor, col=col)
    df[['50a_payor_lon', '50a_payor_lat']] = df.apply(apply_partial, axis=1)
    return df

dfm[['50a_payor_lon', '50a_payor_lat']] = parallel_calc(df, serial_proc_50aPayor, 8, ':50A:')[['50a_payor_lon', '50a_payor_lat']]
# dfm[['50a_payor_lon', '50a_payor_lat']] = serial_proc_50aPayor(df, ':50A:')[['50a_payor_lon', '50a_payor_lat']]

CPU times: user 3.73 s, sys: 1.3 s, total: 5.04 s
Wall time: 56.5 s


In [21]:
dfm

,32a_val_dt,33b_cur,33b_orig_ord_amt,50a_payor_lon,50a_payor_lat
0,2021-08-13,RUB,"1544103,68",/0000000000000000000000000000009621,BIC68DF2C94
1,2021-08-13,RUB,"912070,95",/0000000000000000000000000000007346,BIC68DF2C94
2,2021-08-13,RUB,"21760302,59",/0000000000000000000000000000005305,BIC68DF2C94
3,2021-08-13,RUB,"878587,53",/0000000000000000000000000000004127,BIC68DF2C94
4,2021-08-13,RUB,"6696355,97",/0000000000000000000000000000006978,BIC68DF2C94
...,...,...,...,...,...
379521,2021-08-13,USD,"8339,00",/0000000000000000000000000000003030,BIC6A94AE24
379522,2021-08-13,USD,"4001,56",,
379523,2021-08-13,USD,"4456,00",,
379524,2021-08-13,USD,"8836,10",/0000000000000000000000000000007977,BIC6A94AE24


#### Split 50F - Payor ACCT_NUM, PAYOR_NAME, PAYOR_ADDRESS_LINE_1, PAYOR_ADDRESS_LINE_2 

In [22]:
%%time
def _50fPayor(row, col):
    return pd.Series(re.sub(r"[0-9]?\/", " ", row[col]).split("\n") 
                                            if len(row[col]) > 0 else [''] * 4,
                                            index=['ACCOUNT_NUM', 'NAME', 'ADDRESS_LN_1', 'ADDRESS_LN_2'])

def serial_proc_50fPayor(df, col):
    apply_partial = partial(_50fPayor, col=col)
    df[['50f_payor_acct_nm:', '50f_payor_name:', 
     '50f_payor_add_ln_1', '50f_payor_add_ln_2']] = df.apply(apply_partial, axis=1)
    return df

dfm[['50f_payor_acct_nm:', '50f_payor_name:', 
     '50f_payor_add_ln_1', '50f_payor_add_ln_2']] = parallel_calc(df, serial_proc_50fPayor, 
                                                    8, ':50F:') \
                                                    [['50f_payor_acct_nm:', '50f_payor_name:', 
                                                    '50f_payor_add_ln_1', '50f_payor_add_ln_2']]

# dfm[['50f_payor_acct_nm:', '50f_payor_name:', 
#      '50f_payor_add_ln_1', '50f_payor_add_ln_2']] = serial_proc_50fPayor(df, ':50F:') [['50f_payor_acct_nm:', '50f_payor_name:', 
#                                                                                         '50f_payor_add_ln_1', '50f_payor_add_ln_2']]

CPU times: user 3.73 s, sys: 1.22 s, total: 4.95 s
Wall time: 57.2 s


In [23]:
dfm

,32a_val_dt,33b_cur,33b_orig_ord_amt,50a_payor_lon,50a_payor_lat,50f_payor_acct_nm:,50f_payor_name:,50f_payor_add_ln_1,50f_payor_add_ln_2
0,2021-08-13,RUB,"1544103,68",/0000000000000000000000000000009621,BIC68DF2C94,,,,
1,2021-08-13,RUB,"912070,95",/0000000000000000000000000000007346,BIC68DF2C94,,,,
2,2021-08-13,RUB,"21760302,59",/0000000000000000000000000000005305,BIC68DF2C94,,,,
3,2021-08-13,RUB,"878587,53",/0000000000000000000000000000004127,BIC68DF2C94,,,,
4,2021-08-13,RUB,"6696355,97",/0000000000000000000000000000006978,BIC68DF2C94,,,,
...,...,...,...,...,...,...,...,...,...
379521,2021-08-13,USD,"8339,00",/0000000000000000000000000000003030,BIC6A94AE24,,,,
379522,2021-08-13,USD,"4001,56",,,DRLC US VA 0000008040,EMILIJA MITCHELL,345256 YELLOW BRICK RD,US LINCOLN
379523,2021-08-13,USD,"4456,00",,,DRLC US VA 0000009910,JOO-WON GARCIA,521469 LAKE POINTE AVE,US MIFFLIN
379524,2021-08-13,USD,"8836,10",/0000000000000000000000000000007977,BIC6A94AE24,,,,


#### Convert sender and receiver to their frequecy count

In [24]:
%%time
# dfm['52a_sender'] = df.loc[:, ":52A:"].apply(lambda x: senders_count_dict[x])
dfm['52a_sender'] = df.loc[:, ":52A:"]

CPU times: user 8.51 ms, sys: 150 µs, total: 8.66 ms
Wall time: 8.22 ms


In [25]:
%%time
# dfm['57a_receiver'] = df.loc[:, ":57A:"].apply(lambda x: receivers_count_dict[x])
dfm['57a_receiver'] = df.loc[:, ":57A:"]

CPU times: user 5.72 ms, sys: 3.27 ms, total: 8.98 ms
Wall time: 8.06 ms


In [26]:
def get_geocodes(address):
    try:
        # return coords_dict[address]
        location = coords_dict[address]
        return [location['latitude'], location['longitude']]
    except:
        return ''

In [27]:
%%time

def _50fPayorCoords(row, col):
    return pd.Series(get_geocodes(row[col].strip()), 
                        index=['latitude', 'longitude'])

def serial_proc_50fPayorCoords(df, col):
    apply_partial = partial(_50fPayorCoords, col=col)
    df[['50f_payor_add_lat', '50f_payor_add_lon']] = df.apply(apply_partial, axis=1)
    return df

dfm[['50f_payor_add_lat', 
     '50f_payor_add_lon']] = parallel_calc(dfm, serial_proc_50fPayorCoords, 
                                           8, '50f_payor_add_ln_2') \
                                           [['50f_payor_add_lat', '50f_payor_add_lon']]

CPU times: user 1.4 s, sys: 520 ms, total: 1.92 s
Wall time: 54.2 s


In [28]:
dfm

,32a_val_dt,33b_cur,33b_orig_ord_amt,50a_payor_lon,50a_payor_lat,50f_payor_acct_nm:,50f_payor_name:,50f_payor_add_ln_1,50f_payor_add_ln_2,52a_sender,57a_receiver,50f_payor_add_lat,50f_payor_add_lon
0,2021-08-13,RUB,"1544103,68",/0000000000000000000000000000009621,BIC68DF2C94,,,,,BIC68DF2C94,BIC6A94AE24,,
1,2021-08-13,RUB,"912070,95",/0000000000000000000000000000007346,BIC68DF2C94,,,,,BIC68DF2C94,BIC6A94AE24,,
2,2021-08-13,RUB,"21760302,59",/0000000000000000000000000000005305,BIC68DF2C94,,,,,BIC68DF2C94,BIC6A94AE24,,
3,2021-08-13,RUB,"878587,53",/0000000000000000000000000000004127,BIC68DF2C94,,,,,BIC68DF2C94,BIC6A94AE24,,
4,2021-08-13,RUB,"6696355,97",/0000000000000000000000000000006978,BIC68DF2C94,,,,,BIC68DF2C94,BIC6A94AE24,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
379521,2021-08-13,USD,"8339,00",/0000000000000000000000000000003030,BIC6A94AE24,,,,,BIC6A94AE24,BIC68DF2C94,,
379522,2021-08-13,USD,"4001,56",,,DRLC US VA 0000008040,EMILIJA MITCHELL,345256 YELLOW BRICK RD,US LINCOLN,BIC6A94AE24,BIC68DF2C94,40.8089,-96.7078
379523,2021-08-13,USD,"4456,00",,,DRLC US VA 0000009910,JOO-WON GARCIA,521469 LAKE POINTE AVE,US MIFFLIN,BIC6A94AE24,BIC68DF2C94,37.9412,-81.8221
379524,2021-08-13,USD,"8836,10",/0000000000000000000000000000007977,BIC6A94AE24,,,,,BIC6A94AE24,BIC68DF2C94,,


In [29]:
%%time

def _50kPayorSplit(row, col):
    return pd.Series(re.sub(r"[0-9]?\/", " ", row[col]).split("\n") 
                        if len(row[col]) > 0 else [''] * 4,
                        index=['acct_num', 'name', 'add_ln_1', 'add_ln_2'])

def serial_proc_50kPayorSplit(df, col):
    apply_partial = partial(_50kPayorSplit, col=col)
    df[['50k_payor_acct_num:', '50k_payor_name', 
     '50k_payor_add_ln_1', '50k_payor_add_ln_2']] = df.apply(apply_partial, axis=1)
    return df

dfm[['50k_payor_acct_num:', '50k_payor_name', 
     '50k_payor_add_ln_1', '50k_payor_add_ln_2']] = parallel_calc(df, serial_proc_50kPayorSplit, 
                                                        8, ':50K:') \
                                                        [['50k_payor_acct_num:', '50k_payor_name', 
                                                          '50k_payor_add_ln_1', '50k_payor_add_ln_2']]

CPU times: user 3.54 s, sys: 1.42 s, total: 4.95 s
Wall time: 59.3 s


In [30]:
dfm

,32a_val_dt,33b_cur,33b_orig_ord_amt,50a_payor_lon,50a_payor_lat,50f_payor_acct_nm:,50f_payor_name:,50f_payor_add_ln_1,50f_payor_add_ln_2,52a_sender,57a_receiver,50f_payor_add_lat,50f_payor_add_lon,50k_payor_acct_num:,50k_payor_name,50k_payor_add_ln_1,50k_payor_add_ln_2
0,2021-08-13,RUB,"1544103,68",/0000000000000000000000000000009621,BIC68DF2C94,,,,,BIC68DF2C94,BIC6A94AE24,,,,,,
1,2021-08-13,RUB,"912070,95",/0000000000000000000000000000007346,BIC68DF2C94,,,,,BIC68DF2C94,BIC6A94AE24,,,,,,
2,2021-08-13,RUB,"21760302,59",/0000000000000000000000000000005305,BIC68DF2C94,,,,,BIC68DF2C94,BIC6A94AE24,,,,,,
3,2021-08-13,RUB,"878587,53",/0000000000000000000000000000004127,BIC68DF2C94,,,,,BIC68DF2C94,BIC6A94AE24,,,,,,
4,2021-08-13,RUB,"6696355,97",/0000000000000000000000000000006978,BIC68DF2C94,,,,,BIC68DF2C94,BIC6A94AE24,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379521,2021-08-13,USD,"8339,00",/0000000000000000000000000000003030,BIC6A94AE24,,,,,BIC6A94AE24,BIC68DF2C94,,,,,,
379522,2021-08-13,USD,"4001,56",,,DRLC US VA 0000008040,EMILIJA MITCHELL,345256 YELLOW BRICK RD,US LINCOLN,BIC6A94AE24,BIC68DF2C94,40.8089,-96.7078,,,,
379523,2021-08-13,USD,"4456,00",,,DRLC US VA 0000009910,JOO-WON GARCIA,521469 LAKE POINTE AVE,US MIFFLIN,BIC6A94AE24,BIC68DF2C94,37.9412,-81.8221,,,,
379524,2021-08-13,USD,"8836,10",/0000000000000000000000000000007977,BIC6A94AE24,,,,,BIC6A94AE24,BIC68DF2C94,,,,,,


In [31]:
%%time

def _50kPayorCoords(row, col):
    return pd.Series(get_geocodes(row[col].strip()), 
                        index=['latitude', 'longitude'])

def serial_proc_50kPayorCoords(df, col):
    apply_partial = partial(_50kPayorCoords, col=col)
    df[['50k_payor_add_lat', '50k_payor_add_lon']] = df.apply(apply_partial, axis=1)
    return df

dfm[['50k_payor_add_lat', '50k_payor_add_lon']] = parallel_calc(dfm, serial_proc_50kPayorCoords, 
                                                            8, '50k_payor_add_ln_2') \
                                                            [['50k_payor_add_lat', '50k_payor_add_lon']]

CPU times: user 1.94 s, sys: 668 ms, total: 2.61 s
Wall time: 54.6 s


In [32]:
dfm

,32a_val_dt,33b_cur,33b_orig_ord_amt,50a_payor_lon,50a_payor_lat,50f_payor_acct_nm:,50f_payor_name:,50f_payor_add_ln_1,50f_payor_add_ln_2,52a_sender,57a_receiver,50f_payor_add_lat,50f_payor_add_lon,50k_payor_acct_num:,50k_payor_name,50k_payor_add_ln_1,50k_payor_add_ln_2,50k_payor_add_lat,50k_payor_add_lon
0,2021-08-13,RUB,"1544103,68",/0000000000000000000000000000009621,BIC68DF2C94,,,,,BIC68DF2C94,BIC6A94AE24,,,,,,,,
1,2021-08-13,RUB,"912070,95",/0000000000000000000000000000007346,BIC68DF2C94,,,,,BIC68DF2C94,BIC6A94AE24,,,,,,,,
2,2021-08-13,RUB,"21760302,59",/0000000000000000000000000000005305,BIC68DF2C94,,,,,BIC68DF2C94,BIC6A94AE24,,,,,,,,
3,2021-08-13,RUB,"878587,53",/0000000000000000000000000000004127,BIC68DF2C94,,,,,BIC68DF2C94,BIC6A94AE24,,,,,,,,
4,2021-08-13,RUB,"6696355,97",/0000000000000000000000000000006978,BIC68DF2C94,,,,,BIC68DF2C94,BIC6A94AE24,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379521,2021-08-13,USD,"8339,00",/0000000000000000000000000000003030,BIC6A94AE24,,,,,BIC6A94AE24,BIC68DF2C94,,,,,,,,
379522,2021-08-13,USD,"4001,56",,,DRLC US VA 0000008040,EMILIJA MITCHELL,345256 YELLOW BRICK RD,US LINCOLN,BIC6A94AE24,BIC68DF2C94,40.8089,-96.7078,,,,,,
379523,2021-08-13,USD,"4456,00",,,DRLC US VA 0000009910,JOO-WON GARCIA,521469 LAKE POINTE AVE,US MIFFLIN,BIC6A94AE24,BIC68DF2C94,37.9412,-81.8221,,,,,,
379524,2021-08-13,USD,"8836,10",/0000000000000000000000000000007977,BIC6A94AE24,,,,,BIC6A94AE24,BIC68DF2C94,,,,,,,,


In [33]:
%%time

def _59fbeneficiarySplit(row, col):
    return pd.Series(re.sub(r"[0-9]?\/", " ", row[col]).split("\n") 
                if len(row[col]) > 0 else [''] * 4,
                index=['acct_num', 'name', 'add_ln_1', 'add_ln_2'])

def serial_proc_59fbeneficiarySplit(df, col):
    apply_partial = partial(_59fbeneficiarySplit, col=col)
    df[['59f_ben_acct_num:', '59f_ben_name', 
     '59f_ben_add_ln_1', '59f_ben_add_ln_2']] = df.apply(apply_partial, axis=1)
    return df

dfm[['59f_ben_acct_num:', '59f_ben_name', 
     '59f_ben_add_ln_1', '59f_ben_add_ln_2']] = parallel_calc(df, serial_proc_59fbeneficiarySplit, 
                                                    8, ':59F:') \
                                                    [['59f_ben_acct_num:', '59f_ben_name', 
                                                      '59f_ben_add_ln_1', '59f_ben_add_ln_2']]

CPU times: user 4.02 s, sys: 1.45 s, total: 5.47 s
Wall time: 60 s


In [34]:
dfm

,32a_val_dt,33b_cur,33b_orig_ord_amt,50a_payor_lon,50a_payor_lat,50f_payor_acct_nm:,50f_payor_name:,50f_payor_add_ln_1,50f_payor_add_ln_2,52a_sender,...,50k_payor_acct_num:,50k_payor_name,50k_payor_add_ln_1,50k_payor_add_ln_2,50k_payor_add_lat,50k_payor_add_lon,59f_ben_acct_num:,59f_ben_name,59f_ben_add_ln_1,59f_ben_add_ln_2
0,2021-08-13,RUB,"1544103,68",/0000000000000000000000000000009621,BIC68DF2C94,,,,,BIC68DF2C94,...,,,,,,,0000000000000000000000000000006099,ANGEL JAMES,1789 PHILLIPS RD,US GLENDALE
1,2021-08-13,RUB,"912070,95",/0000000000000000000000000000007346,BIC68DF2C94,,,,,BIC68DF2C94,...,,,,,,,0000000000000000000000000000000874,JAKUB EVANS,369653 PEAR ST,US WELLS
2,2021-08-13,RUB,"21760302,59",/0000000000000000000000000000005305,BIC68DF2C94,,,,,BIC68DF2C94,...,,,,,,,0000000000000000000000000000005436,ZIGA MITCHELL,131041 DEBRA DR,US GROVER
3,2021-08-13,RUB,"878587,53",/0000000000000000000000000000004127,BIC68DF2C94,,,,,BIC68DF2C94,...,,,,,,,0000000000000000000000000000008707,YAEL DEMERS,50438 HAYMARKET ST,US MOORE
4,2021-08-13,RUB,"6696355,97",/0000000000000000000000000000006978,BIC68DF2C94,,,,,BIC68DF2C94,...,,,,,,,0000000000000000000000000000004326,NOEL PLANTE,43790 MOSHER CT,US WAUCHULA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379521,2021-08-13,USD,"8339,00",/0000000000000000000000000000003030,BIC6A94AE24,,,,,BIC6A94AE24,...,,,,,,,0000000000000000000000000000003627,JESSE LEFEBVRE,147085 CAPITOL AVE,US SOUTH SAN FRANCISCO
379522,2021-08-13,USD,"4001,56",,,DRLC US VA 0000008040,EMILIJA MITCHELL,345256 YELLOW BRICK RD,US LINCOLN,BIC6A94AE24,...,,,,,,,0000000000000000000000000000009377,SULTAN OZTURK,120952 PONDEROSA DR,US WARETOWN
379523,2021-08-13,USD,"4456,00",,,DRLC US VA 0000009910,JOO-WON GARCIA,521469 LAKE POINTE AVE,US MIFFLIN,BIC6A94AE24,...,,,,,,,0000000000000000000000000000001708,DIMITRIOS PASCUAL,134541 E NORTHGATE CT,US COMMERCE CITY
379524,2021-08-13,USD,"8836,10",/0000000000000000000000000000007977,BIC6A94AE24,,,,,BIC6A94AE24,...,,,,,,,0000000000000000000000000000005070,ADAMA YILDIZ,55827 PRESERVATION WAY,US ESTES PARK


In [35]:
%%time

def _59fBeneficiaryCoords(row, col):
    return pd.Series(get_geocodes(row[col].strip()), 
                        index=['latitude', 'longitude'])

def serial_proc_59fBeneficiaryCoords(df, col):
    apply_partial = partial(_59fBeneficiaryCoords, col=col)
    df[['59f_ben_add_lat', '59f_ben_add_lon']] = df.apply(apply_partial, axis=1)
    return df

dfm[['59f_ben_add_lat', '59f_ben_add_lon']] = parallel_calc(dfm, serial_proc_59fBeneficiaryCoords, 
                                                           8, '59f_ben_add_ln_2') \
                                                           [['59f_ben_add_lat', '59f_ben_add_lon']]


CPU times: user 3.07 s, sys: 1e+03 ms, total: 4.07 s
Wall time: 53.5 s


In [36]:
dfm

,32a_val_dt,33b_cur,33b_orig_ord_amt,50a_payor_lon,50a_payor_lat,50f_payor_acct_nm:,50f_payor_name:,50f_payor_add_ln_1,50f_payor_add_ln_2,52a_sender,...,50k_payor_add_ln_1,50k_payor_add_ln_2,50k_payor_add_lat,50k_payor_add_lon,59f_ben_acct_num:,59f_ben_name,59f_ben_add_ln_1,59f_ben_add_ln_2,59f_ben_add_lat,59f_ben_add_lon
0,2021-08-13,RUB,"1544103,68",/0000000000000000000000000000009621,BIC68DF2C94,,,,,BIC68DF2C94,...,,,,,0000000000000000000000000000006099,ANGEL JAMES,1789 PHILLIPS RD,US GLENDALE,34.1469,-118.248
1,2021-08-13,RUB,"912070,95",/0000000000000000000000000000007346,BIC68DF2C94,,,,,BIC68DF2C94,...,,,,,0000000000000000000000000000000874,JAKUB EVANS,369653 PEAR ST,US WELLS,47.5664,-99.6509
2,2021-08-13,RUB,"21760302,59",/0000000000000000000000000000005305,BIC68DF2C94,,,,,BIC68DF2C94,...,,,,,0000000000000000000000000000005436,ZIGA MITCHELL,131041 DEBRA DR,US GROVER,42.7956,-110.929
3,2021-08-13,RUB,"878587,53",/0000000000000000000000000000004127,BIC68DF2C94,,,,,BIC68DF2C94,...,,,,,0000000000000000000000000000008707,YAEL DEMERS,50438 HAYMARKET ST,US MOORE,35.8156,-101.91
4,2021-08-13,RUB,"6696355,97",/0000000000000000000000000000006978,BIC68DF2C94,,,,,BIC68DF2C94,...,,,,,0000000000000000000000000000004326,NOEL PLANTE,43790 MOSHER CT,US WAUCHULA,27.5473,-81.8115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379521,2021-08-13,USD,"8339,00",/0000000000000000000000000000003030,BIC6A94AE24,,,,,BIC6A94AE24,...,,,,,0000000000000000000000000000003627,JESSE LEFEBVRE,147085 CAPITOL AVE,US SOUTH SAN FRANCISCO,37.6535,-122.417
379522,2021-08-13,USD,"4001,56",,,DRLC US VA 0000008040,EMILIJA MITCHELL,345256 YELLOW BRICK RD,US LINCOLN,BIC6A94AE24,...,,,,,0000000000000000000000000000009377,SULTAN OZTURK,120952 PONDEROSA DR,US WARETOWN,39.7939,-74.2232
379523,2021-08-13,USD,"4456,00",,,DRLC US VA 0000009910,JOO-WON GARCIA,521469 LAKE POINTE AVE,US MIFFLIN,BIC6A94AE24,...,,,,,0000000000000000000000000000001708,DIMITRIOS PASCUAL,134541 E NORTHGATE CT,US COMMERCE CITY,39.8083,-104.934
379524,2021-08-13,USD,"8836,10",/0000000000000000000000000000007977,BIC6A94AE24,,,,,BIC6A94AE24,...,,,,,0000000000000000000000000000005070,ADAMA YILDIZ,55827 PRESERVATION WAY,US ESTES PARK,40.3664,-105.503


In [37]:
dfm['71A_chg_dtls'] = df[":71A:"]

In [38]:
%%time

def _71fCurAmt(row, col):
    return pd.Series(re.split("(^[A-Z][A-Z][A-Z])([0-9]*,[0-9]*)", row[col])[1:3] 
                     if len(row[col]) > 0 else [''] * 2, 
                     index=['cur', 'amt'])

def serial_proc_71fCurAmt(df, col):
    apply_partial = partial(_71fCurAmt, col=col)
    df[["71f_chg_dtls_cur", "71f_chg_dtls_amt"]] = df.apply(apply_partial, axis=1)
    return df

dfm[["71f_chg_dtls_cur", "71f_chg_dtls_amt"]] = parallel_calc(df, serial_proc_71fCurAmt, 8, ':71F:') \
                                                        [["71f_chg_dtls_cur", "71f_chg_dtls_amt"]]

CPU times: user 3.81 s, sys: 1.22 s, total: 5.03 s
Wall time: 57.7 s


In [39]:
%%time
dfm['33b_orig_ord_amt'] = pd.to_numeric(dfm['33b_orig_ord_amt'].apply(lambda x: x.replace(',', '.')))
dfm['71f_chg_dtls_amt'] = pd.to_numeric(dfm['71f_chg_dtls_amt'].apply(lambda x: x.replace(',', '.')))

CPU times: user 336 ms, sys: 4.32 ms, total: 341 ms
Wall time: 340 ms


In [40]:
dfm

,32a_val_dt,33b_cur,33b_orig_ord_amt,50a_payor_lon,50a_payor_lat,50f_payor_acct_nm:,50f_payor_name:,50f_payor_add_ln_1,50f_payor_add_ln_2,52a_sender,...,50k_payor_add_lon,59f_ben_acct_num:,59f_ben_name,59f_ben_add_ln_1,59f_ben_add_ln_2,59f_ben_add_lat,59f_ben_add_lon,71A_chg_dtls,71f_chg_dtls_cur,71f_chg_dtls_amt
0,2021-08-13,RUB,1544103.68,/0000000000000000000000000000009621,BIC68DF2C94,,,,,BIC68DF2C94,...,,0000000000000000000000000000006099,ANGEL JAMES,1789 PHILLIPS RD,US GLENDALE,34.1469,-118.248,SHA,,NaN
1,2021-08-13,RUB,912070.95,/0000000000000000000000000000007346,BIC68DF2C94,,,,,BIC68DF2C94,...,,0000000000000000000000000000000874,JAKUB EVANS,369653 PEAR ST,US WELLS,47.5664,-99.6509,BEN,RUB,0.0
2,2021-08-13,RUB,21760302.59,/0000000000000000000000000000005305,BIC68DF2C94,,,,,BIC68DF2C94,...,,0000000000000000000000000000005436,ZIGA MITCHELL,131041 DEBRA DR,US GROVER,42.7956,-110.929,SHA,,NaN
3,2021-08-13,RUB,878587.53,/0000000000000000000000000000004127,BIC68DF2C94,,,,,BIC68DF2C94,...,,0000000000000000000000000000008707,YAEL DEMERS,50438 HAYMARKET ST,US MOORE,35.8156,-101.91,SHA,,NaN
4,2021-08-13,RUB,6696355.97,/0000000000000000000000000000006978,BIC68DF2C94,,,,,BIC68DF2C94,...,,0000000000000000000000000000004326,NOEL PLANTE,43790 MOSHER CT,US WAUCHULA,27.5473,-81.8115,SHA,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379521,2021-08-13,USD,8339.00,/0000000000000000000000000000003030,BIC6A94AE24,,,,,BIC6A94AE24,...,,0000000000000000000000000000003627,JESSE LEFEBVRE,147085 CAPITOL AVE,US SOUTH SAN FRANCISCO,37.6535,-122.417,SHA,,NaN
379522,2021-08-13,USD,4001.56,,,DRLC US VA 0000008040,EMILIJA MITCHELL,345256 YELLOW BRICK RD,US LINCOLN,BIC6A94AE24,...,,0000000000000000000000000000009377,SULTAN OZTURK,120952 PONDEROSA DR,US WARETOWN,39.7939,-74.2232,BEN,USD,0.0
379523,2021-08-13,USD,4456.00,,,DRLC US VA 0000009910,JOO-WON GARCIA,521469 LAKE POINTE AVE,US MIFFLIN,BIC6A94AE24,...,,0000000000000000000000000000001708,DIMITRIOS PASCUAL,134541 E NORTHGATE CT,US COMMERCE CITY,39.8083,-104.934,OUR,,NaN
379524,2021-08-13,USD,8836.10,/0000000000000000000000000000007977,BIC6A94AE24,,,,,BIC6A94AE24,...,,0000000000000000000000000000005070,ADAMA YILDIZ,55827 PRESERVATION WAY,US ESTES PARK,40.3664,-105.503,SHA,,NaN


In [41]:
end = time.perf_counter()

In [42]:
print(f"Total time elasped : {(end - start)/60:.2f} minutes !!")

Total time elasped : 10.69 minutes !!


In [43]:
%%time
dfm.to_csv('data/hackathon_parsed_format_a.csv', index = False)

CPU times: user 5.76 s, sys: 74.8 ms, total: 5.84 s
Wall time: 5.84 s
